In [1]:
"""
Update if [issue]
[Var],[Feature]

"""

'\nUpdate if [issue]\n[Var],[Feature]\n\n'

In [2]:
#REMEBER update to "VERSION" b4 submit!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
emulation_mode = False # Submit version!
#emulation_mode =True

#REMEBER update model to needed VERSION b4 submit!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
model_perfix = "less_leaf"

#all avaible variable
feature_cols = [
        'date_hvGame', 'date_MajorGame', 'isActive_player',
    #player in game Stats
       'battingOrder', 'gamesPlayedBatting', 'flyOuts', 'groundOuts',
       'runsScored', 'doubles', 'triples', 'homeRuns', 'strikeOuts',
       'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch', 'atBats',
       'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
       'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
       'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
       'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
       'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
       'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
       'groundOutsPitching', 'runsPitching', 'doublesPitching',
       'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
       'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
       'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
       'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
       'earnedRuns', 'battersFaced', 'outsPitching',
       'pitchesThrown', 'balls', 'strikes', 'hitBatsmen', 'balks',
       'wildPitches', 'pickoffsPitching', 'rbiPitching',
       'gamesFinishedPitching', 'inheritedRunners', 'inheritedRunnersScored',
       'catchersInterferencePitching', 'sacBuntsPitching', 'sacFliesPitching',
       'saves', 'holds', 'blownSaves', 'assists', 'putOuts', 'errors',
       'chances', 
    #team standing Stats
    'divisionRank', 'leagueRank', 'wildCardRank',
       'wins',
       'losses', 'pct', 'runsAllowed', 'runsScored_Team', 'homeWins', 'homeLosses', 'awayWins',
       'awayLosses', 'lastTenWins', 'lastTenLosses', 'extraInningWins',
       'extraInningLosses', 'oneRunWins', 'oneRunLosses', 'divWins',
       'divLosses', 'alWins', 'alLosses', 'nlWins', 'nlLosses', 'xWinLossPct',
       'Streak_count','firstday_confirm_division','confirm_division_heat',
    #Game info
       'gameNumber', 'scheduledInnings',
       'gamesInSeries',  'teamLosses',
       'teamWinPct', 'teamWins', 'teamScore',  'oppWins',
       'oppLosses', 'oppWinPct',  'oppScore', 
       'home_away', 
    #Award gain
    'Award_1stword_count','Award_Level_max']
    
#all Category feature
Cat_feature = ['playerId','statusCode','positionCode','teamId','positionType',
               'typeCode_Transit','toTeamId_Transit','fromTeamId_Transit',
               'codedGameState', 'detailedGameState','gameType','oppId','dayNight','teamWinner','oppWinner',
              'StreakW','divisionId','doubleHeader',
              'Award_1stword_sum',
              'teamId_lastGameinMLB','roster_status_lastGame']

T1_fea = ['target1_median_inY_S','target1_std_inY_S']#,'target1_preiod_1less']
T2_fea = ['target2_10log100_median_inY_S', 'target2_10log100_std_inY_S']#,'target2_10log100_preiod_1less']
T3_fea = ['target3_median_inY_S','target3_std_inY_S']#,'target3_preiod_1less']
T4_fea = ['target4_10log100_median_inY_S','target4_10log100_std_inY_S']#,'target4_10log100_preiod_1less']


In [3]:

# Import Modules
import numpy as np
import pandas as pd
import pickle
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas
import time
import datetime
pd.set_option('display.max_columns', None)

from tqdm import tqdm
import lightgbm as lgbm
import mlb
import gc

import warnings
from typing import Optional, Tuple
class Environment:
    def __init__(self,
                 data_dir: str,
                 eval_start_day: int,
                 eval_end_day: Optional[int],
                 use_updated: bool,
                 multiple_days_per_iter: bool):
        warnings.warn('this is mock module for mlb')

        postfix = '_updated' if use_updated else ''
        
        # recommend to replace this with pickle, feather etc to speedup preparing data
        df_train = pd.read_csv(os.path.join(data_dir, f'train{postfix}.csv'))

        players = pd.read_csv(os.path.join(data_dir, 'players.csv'))

        self.players = players[players['playerForTestSetAndFuturePreds'] == True]['playerId'].astype(str)
        if eval_end_day is not None:
            self.df_train = df_train.set_index('date').loc[eval_start_day:eval_end_day]
        else:
            self.df_train = df_train.set_index('date').loc[eval_start_day:]
        self.date = self.df_train.index.values
        self.n_rows = len(self.df_train)
        self.multiple_days_per_iter = multiple_days_per_iter

        assert self.n_rows > 0, 'no data to emulate'

    def predict(self, df: pd.DataFrame) -> None:
        # if you want to emulate public LB, store your prediction here and calculate MAE
        pass

    def iter_test(self) -> Tuple[pd.DataFrame, pd.DataFrame]:
        if self.multiple_days_per_iter:
            for i in range(self.n_rows // 2):
                date1 = self.date[2 * i]
                date2 = self.date[2 * i + 1]
                sample_sub1 = self._make_sample_sub(date1)
                sample_sub2 = self._make_sample_sub(date2)
                sample_sub = pd.concat([sample_sub1, sample_sub2]).reset_index(drop=True)
                df = self.df_train.loc[date1:date2]

                yield df, sample_sub.set_index('date')
        else:
            for i in range(self.n_rows):
                date = self.date[i]
                sample_sub = self._make_sample_sub(date)
                df = self.df_train.loc[date:date]

                yield df, sample_sub.set_index('date')

    def _make_sample_sub(self, date: int) -> pd.DataFrame:
        next_day = (pd.to_datetime(date, format='%Y%m%d') + pd.to_timedelta(1, 'd')).strftime('%Y%m%d')
        sample_sub = pd.DataFrame()
        sample_sub['date_playerId'] = next_day + '_' + self.players
        sample_sub['target1'] = 0
        sample_sub['target2'] = 0
        sample_sub['target3'] = 0
        sample_sub['target4'] = 0
        sample_sub['date'] = date
        return sample_sub


class MLBEmulator:
    def __init__(self,
                 data_dir: str = '../input/mlb-player-digital-engagement-forecasting',
                 eval_start_day: int = 20210401,
                 eval_end_day: Optional[int] = 20210430,
                 use_updated: bool = True,
                 multiple_days_per_iter: bool = False):
        self.data_dir = data_dir
        self.eval_start_day = eval_start_day
        self.eval_end_day = eval_end_day
        self.use_updated = use_updated
        self.multiple_days_per_iter = multiple_days_per_iter

    def make_env(self) -> Environment:
        return Environment(self.data_dir,
                           self.eval_start_day,
                           self.eval_end_day,
                           self.use_updated,
                           self.multiple_days_per_iter)

    
def target_10log100(x):
    return np.log10(x * 100+1)
def target_10log100_reverse(x):
    return (10**x -1)/100


def take_data(test_df,Table,player_list,col_list):
    #if NaN, will gen NaN data in full player and full columns in varible
    if test_df[Table].iloc[0] == test_df[Table].iloc[0]:
        selected_df = pd.read_json(test_df[Table].iloc[0])
    else: # Dealing with missing values i.e NaN
        selected_df = pd.DataFrame({'playerId': player_list})
        for col in col_list:
            if col == 'playerId': continue
            selected_df[col] = np.nan
    return selected_df

def take_data_Team(test_df,Table,team_list,col_list):
    #if NaN, will gen NaN data in full player and full columns in varible
    if test_df[Table].iloc[0] == test_df[Table].iloc[0]:
        selected_df = pd.read_json(test_df[Table].iloc[0])
    else: # Dealing with missing values i.e NaN
        selected_df = pd.DataFrame({'teamId': team_list})
        for col in col_list:
            if col == 'teamId': continue
            selected_df[col] = np.nan
    return selected_df

def reduce_mem_usage(train_data):
    for col in train_data.columns:
        col_type = train_data[col].dtype
        if col_type != object:
            c_min = train_data[col].min()
            c_max = train_data[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    train_data[col] = train_data[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    train_data[col] = train_data[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    train_data[col] = train_data[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    train_data[col] = train_data[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    train_data[col] = train_data[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    train_data[col] = train_data[col].astype(np.float32)
                else:
                    train_data[col] = train_data[col].astype(np.float64)
        else:
            train_data[col] = train_data[col].astype('category')

        end_mem = train_data.memory_usage().sum() / 1024**2
        print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
        print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

        return train_data

"""
def return_default_value_if_fails(default_value):

    def decorator(func):
        def inner(*args, **kwargs):
            try:
                return func(*args, **kwargs)
            except Exception as e:
                fails.append((func, (args, kwargs), e))
                return default_value
        return inner

    return decorator
def _parse_player_height(self):


        @return_default_value_if_fails(default_value=75)
        def parse(x):
            return 12 * int(x.split('-')[0]) + int(x.split('-')[1])
        return self.df['PlayerHeight'].apply(parse)
"""

"\ndef return_default_value_if_fails(default_value):\n\n    def decorator(func):\n        def inner(*args, **kwargs):\n            try:\n                return func(*args, **kwargs)\n            except Exception as e:\n                fails.append((func, (args, kwargs), e))\n                return default_value\n        return inner\n\n    return decorator\ndef _parse_player_height(self):\n\n\n        @return_default_value_if_fails(default_value=75)\n        def parse(x):\n            return 12 * int(x.split('-')[0]) + int(x.split('-')[1])\n        return self.df['PlayerHeight'].apply(parse)\n"

In [4]:
filepath = '/kaggle/input/mlb-ms' 
# 1 base Stat - median, std etc
Player_meadian_year = pd.read_csv(filepath + '/Player_meadian_year.csv' ,index_col = 0)
team_list =  pd.read_csv('/kaggle/input/mlb-player-digital-engagement-forecasting' + '/teams.csv' )
team_list = team_list['id']
# 2 Cat Mapping from model
file_dict = open(filepath +'/Cat_feature_MapDict.pkl',"rb")
Cat_feature_MapDict =  pickle.load(file_dict)

# 3 >2021 Jun Jul training target
Train_L_set = pd.read_csv(filepath + '/Train_L_set.csv',index_col = 0 )

# 4 Varible column in Train/test for aviod NaN issue
file_dict = open(filepath +'/item_feature_MapDict.pkl',"rb")
item_feature_MapDict =  pickle.load(file_dict)
# 5 load model

for i in range(4):
    globals()['model'+str(i+1)] = lgbm.Booster(model_file=filepath + '/' +'lgbr_'+model_perfix +'_'+str(i+1)+'.mdl')


/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (174,175,176,177,180,181,184,189,195,197,199,200) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
Train_L_set

,Unnamed: 0.1,index,engagementMetricsDate,playerId,target1,target2,target3,target4,Date_DT,Date_pre_DT,Date_pre_Year,in_season,date_hvGame,date_MajorGame,gameDate_DT_x,gameDate_DT_y,statusCode,teamId,teamId_firmed,teamId_lastGameinMLB,roster_status_lastGame,isActive_player,isActiveMajorGame_player,target1_10log100,target2_10log100,target3_10log100,target4_10log100,target1_median_inY_S,target1_std_inY_S,target2_10log100_median_inY_S,target2_10log100_std_inY_S,target3_median_inY_S,target3_std_inY_S,target4_10log100_median_inY_S,target4_10log100_std_inY_S,T1_abnormal,T3_abnormal,T1and3_abnormal,T1_abnormal_value,T3_abnormal_value,Date_DT_preiod_1less,target1_preiod_1less,target2_10log100_preiod_1less,target3_preiod_1less,target4_10log100_preiod_1less,fromTeamId_Transit,toTeamId_Transit,typeCode_Transit,date_DT,gamePk,positionCode,teamId_Player,positionName,positionType,battingOrder,gamesPlayedBatting,flyOuts,groundOuts,runsScored,doubles,triples,homeRuns,strikeOuts,baseOnBalls,intentionalWalks,hits,hitByPitch,atBats,caughtStealing,stolenBases,groundIntoDoublePlay,groundIntoTriplePlay,plateAppearances,totalBases,rbi,leftOnBase,sacBunts,sacFlies,catchersInterference,pickoffs,gamesPlayedPitching,gamesStartedPitching,completeGamesPitching,shutoutsPitching,winsPitching,lossesPitching,flyOutsPitching,airOutsPitching,groundOutsPitching,runsPitching,doublesPitching,triplesPitching,homeRunsPitching,strikeOutsPitching,baseOnBallsPitching,intentionalWalksPitching,hitsPitching,hitByPitchPitching,atBatsPitching,caughtStealingPitching,stolenBasesPitching,inningsPitched,saveOpportunities,earnedRuns,battersFaced,outsPitching,pitchesThrown,balls,strikes,hitBatsmen,balks,wildPitches,pickoffsPitching,rbiPitching,gamesFinishedPitching,inheritedRunners,inheritedRunnersScored,catchersInterferencePitching,sacBuntsPitching,sacFliesPitching,saves,holds,blownSaves,assists,putOuts,errors,chances,gameDate_DT,gameDate,divisionId,teamId_Team,teamName,streakCode,divisionRank,leagueRank,wildCardRank,leagueGamesBack,sportGamesBack,divisionGamesBack,wins,losses,pct,runsAllowed,runsScored_Team,divisionChamp,divisionLeader,wildCardLeader,eliminationNumber,wildCardEliminationNumber,homeWins,homeLosses,awayWins,awayLosses,lastTenWins,lastTenLosses,extraInningWins,extraInningLosses,oneRunWins,oneRunLosses,divWins,divLosses,alWins,alLosses,nlWins,nlLosses,xWinLossPct,gameDate_DT_Team,gameDate_year,gameDate_month,StreakW,Streak_count,firstday_confirm_division,confirm_division_heat,gameType,gameDate_Game,codedGameState,detailedGameState,isTie,gameNumber,doubleHeader,dayNight,scheduledInnings,gamesInSeries,seriesDescription,teamId_Game,teamWins,teamLosses,teamWinPct,teamWinner,teamScore,oppId,oppWins,oppLosses,oppWinPct,oppWinner,oppScore,gameDate_DT_Game,home_away,awardDate_DT,Award_1stword_sum,Award_1stword_count,Award_Level_max,label_playerId,label_statusCode,label_positionCode,label_teamId,label_positionType,label_typeCode_Transit,label_toTeamId_Transit,label_fromTeamId_Transit,label_codedGameState,label_detailedGameState,label_gameType,label_oppId,label_dayNight,label_teamWinner,label_oppWinner,label_StreakW,label_divisionId,label_doubleHeader,label_Award_1stword_sum,label_teamId_lastGameinMLB,label_roster_status_lastGame
525627,2444346,0,2021-04-02,624415,2.810640,3.998462,0.136137,10.012438,2021-04-02,2021-04-01,2021,True,True,True,2021-04-01,2021-04-01,A,141.0,141.0,141.0,A,True,True,2.450348,2.602978,1.164760,3.000973,0.328060,4.917802,2.671544,0.351191,0.068066,5.122582,2.617631,0.384535,False,False,False,0.0,0.0,2021-04-01,9.255878,3.425394,0.079239,3.264147,NaN,NaN,NaN,NaN,634642.0,5.0,141.0,Third Base,Infielder,200.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,0.0,0.0,5.0,2021-04-01,2021-04-01,201.0,141.0,Toronto Blue Jays,W1,2.0,7

In [6]:

Cat_feature_pd = pd.DataFrame(Cat_feature)
Cat_feature_pd = "label_" +Cat_feature_pd 
Cat_feature_label = list(Cat_feature_pd[0].values)

feature_cols.extend(Cat_feature_label)
feature_colsM1 = list(feature_cols)
feature_colsM2 = list(feature_cols)
feature_colsM3 = list(feature_cols)
feature_colsM4 = list(feature_cols)

feature_colsM1.extend(T1_fea)
feature_colsM2.extend(T2_fea)
feature_colsM3.extend(T3_fea)
feature_colsM4.extend(T4_fea)

#feature_cols.extend(T1_fea)
#feature_cols.extend(T2_fea)
#feature_cols.extend(T3_fea)
#feature_cols.extend(T4_fea)

"""

"""
review_items_in_player = ['gamePk','positionCode','teamId',
       'positionName', 'positionType', 'battingOrder', 'gamesPlayedBatting',
       'flyOuts', 'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
       'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
       'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
       'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
       'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
       'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
       'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
       'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
       'groundOutsPitching', 'runsPitching', 'doublesPitching',
       'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
       'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
       'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
       'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
       'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
       'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
       'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
       'inheritedRunnersScored', 'catchersInterferencePitching',
       'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
       'assists', 'putOuts', 'errors', 'chances']
standing_list = [ 'divisionId', 'teamId', 'teamName',
       'streakCode', 'divisionRank', 'leagueRank', 'wildCardRank',
       'leagueGamesBack', 'sportGamesBack', 'divisionGamesBack', 'wins',
       'losses', 'pct', 'runsAllowed', 'runsScored', 'divisionChamp',
       'divisionLeader', 'wildCardLeader', 'eliminationNumber',
       'wildCardEliminationNumber', 'homeWins', 'homeLosses', 'awayWins',
       'awayLosses', 'lastTenWins', 'lastTenLosses', 'extraInningWins',
       'extraInningLosses', 'oneRunWins', 'oneRunLosses', 'divWins', 'divLosses', 'alWins', 'alLosses',
       'nlWins', 'nlLosses', 'xWinLossPct']


review_items_in_player.extend(['playerId'])

award_known_1stword = ['Uniform', 'ALPB', 'INT', 'FSL', 'Baseball', 'MLB', 'AZL',
           'MEX',  'World', 'NL', 'AL', 'GCL', 'APP', 'DWL',
           'PWL', 'CAR', 'Mult', 'VWL', 'SAL',  'SOU', 'PCL', 'CAL',
            'All-MLB','AFL', 'MLBPAA', 'NLCS','DSL', 'TEX', 'EAS', 
           'MID',  'ALCS', 'MiLB','NWL', 'PIO','NYP',  'USA', ]

In [7]:
aa = pd.DataFrame(Cat_feature_MapDict['teamId'].keys(),columns=['value'])
aa = list(Cat_feature_MapDict['teamId'].keys())
kk = aa[6]

#aa['TF'] = aa.isna()
#aa[aa['TF']==True]['value'].iloc[0]
Cat_feature_MapDict['teamId'][kk]
Cat_feature_MapDict['doubleHeader']

{'NA': 0, 'N': 1, 'Y': 2, 'S': 3}

In [8]:
Train_L_set_save =pd.concat([pd.DataFrame(),Train_L_set])

In [9]:
#uncomment for Stop loading!

"""

"""
acc_record_target = pd.DataFrame()
acc_record_x = pd.DataFrame()
Train_L_set =  pd.concat([pd.DataFrame(),Train_L_set_save])
Cat_feature = Cat_feature_MapDict.keys()
#Cat_feature = ['playerId', 'statusCode']
player_item = [  'rosters', 'transactions','awards','playerBoxScores']
team_item = [ 'games',  'standings',]
player_list = Player_meadian_year['playerId'].unique() #[issue]
team_list = team_list
first_data_swtich = True

if emulation_mode:
    mlb = MLBEmulator(eval_start_day=20210501, eval_end_day=20210520) #20210501
    #mlb = MLBEmulator(eval_start_day=20210712, eval_end_day=20210712) #20210501
else:
    import mlb

env = mlb.make_env() # initialize the environment
iter_test = env.iter_test()

# iterator which loops over each date in test set
for (test_df, sample_prediction_df) in iter_test:
    #date[0]
    
    app_date = pd.to_datetime(test_df.index[0],format="%Y%m%d")
    print(app_date)
    Submit_index = sample_prediction_df.index
    columns_store = sample_prediction_df.columns
    
#Json to CSV
    for Table in player_item:
        globals()['app_'+Table] = take_data(test_df,Table,player_list,item_feature_MapDict['train_'+Table])
    for Table in team_item:
        globals()['app_'+Table] = take_data_Team(test_df,Table,team_list,item_feature_MapDict['train_'+Table])
    
#Set list for all players
    player_df = pd.DataFrame(player_list,columns = ['playerId'])
    gc.collect()
#[feature] in_season or not [1]
    player_df['in_season']=True #Assume all test date are in season preiod, where data after May as it end at Sep

#Set selected train data [now 2019,2021 & in season ]
    #no need

#[feature] in_game preiod [2]
    player_df['date_hvGame'] = (pd.isna(test_df['games'])==0).iloc[0] #[issue]

#[feature] in_Major game preiod "R","D","F","L","W","C","N","P","A" [3]
    Major_G = list(["R","D","F","L","W","C","N","P","A"])
    player_df['date_MajorGame'] = (pd.isna(test_df['games'])==0).iloc[0] #[issue]

#[feature] player in_game stutas at preiod[4]
#[Var] isActive_player[5]
    player_df = pd.merge(player_df,app_rosters,left_on=['playerId'], right_on=['playerId'] ,how = "left")
    player_df['isActive_player'] = player_df['statusCode'] == "A"
#[Var]Acc info in teamId & roster status
    if first_data_swtich == True :
        last_recorded_Cat = pd.concat([pd.DataFrame(),
                                       Train_L_set[Train_L_set['Date_pre_DT'] == app_date][['playerId','teamId_lastGameinMLB','roster_status_lastGame']]])
        if len(last_recorded_Cat) == 0: #sub in median in inSeason 2021 if cannot find the related date
            last_recorded_Cat = pd.concat([pd.DataFrame(),
                                           Train_L_set[Train_L_set['Date_pre_DT'] == Train_L_set['Date_pre_DT'].max()][['playerId','teamId_lastGameinMLB','roster_status_lastGame']]])
    
    player_df= pd.merge(player_df, 
                        last_recorded_Cat, 
                        left_on=['playerId'], right_on=['playerId'] ,how = "left") #''suffixes=('', '_preiod_1less')
    
    player_df['teamId_lastGameinMLB'] = player_df['teamId_lastGameinMLB'].replace("NA",np.nan)
    #player_df['teamId_lastGameinMLB'] = player_df['teamId_lastGameinMLB'].astype(int)
    player_df['teamId_lastGameinMLB'] = player_df.apply(lambda x: x['teamId'] if x['teamId'] >0 else x['teamId_lastGameinMLB'],axis=1)
    player_df['roster_status_lastGame'] = player_df.apply(lambda x: x['statusCode'] if pd.isnull(x['statusCode'])==0 else x['roster_status_lastGame'],axis=1)
   
    
#[feature] Active player in_game at MajorGame
    #no Used

#[targets] 10log100 in targets
    #no Used 

#[Var]Season/nonSeason Target Median in date's year
    apply_logic_intest = (Player_meadian_year['Date_pre_Year'] == 2021)&(Player_meadian_year['in_season'] == True)
    player_df = pd.merge(player_df,Player_meadian_year[apply_logic_intest],left_on=['playerId'], right_on=['playerId'] ,how = "left")

#[Var] T1,3 abnormal, 1.282 for <90% in N(0,1)
    #only use in training

#[Var] targets in time preiod t-1
    if first_data_swtich ==True: 
        Train_L_set = Train_L_set[Train_L_set['Date_pre_DT'] == app_date - datetime.timedelta(days=1)][['playerId','target1','target2_10log100','target3','target4_10log100']]
        if len(Train_L_set) == 0: #sub in median in inSeason 2021 if cannot find the related date
            Train_L_set = pd.DataFrame({'playerId': player_list})
            Train_L_set = pd.merge(Train_L_set,Player_meadian_year[apply_logic_intest][['playerId','target1_median_inY_S','target2_10log100_median_inY_S','target3_median_inY_S','target4_10log100_median_inY_S']],left_on=['playerId'], right_on=['playerId'] ,how = "left")
    else:
        #[Assuming] consetive date in iter_test
        Train_L_set= pd.concat([pd.DataFrame(),last_recorded_target_set[['playerId','target1','target2_10log100','target3','target4_10log100']]]) 
    
    #adjsut columns names
    Train_L_set.columns = ['playerId','target1_preiod_1less','target2_10log100_preiod_1less','target3_preiod_1less','target4_10log100_preiod_1less']
    #Train_L_set = train_nextDayPlayerEngagement[train_nextDayPlayerEngagement['Date_pre_DT'] == train_nextDayPlayerEngagement['Date_pre_DT'].max()]
    player_df= pd.merge(player_df, 
                        Train_L_set[['playerId','target1_preiod_1less','target2_10log100_preiod_1less','target3_preiod_1less','target4_10log100_preiod_1less']], 
                        left_on=['playerId'], right_on=['playerId'] ,how = "left") #''suffixes=('', '_preiod_1less')
    print('1player',len(player_df))
#[feature] transition
    app_transactions = app_transactions[['playerId','fromTeamId','toTeamId','typeCode']]
    app_transactions.columns = ['playerId','fromTeamId_Transit','toTeamId_Transit','typeCode_Transit']
    app_transactions['playerId'] = app_transactions['playerId'].fillna(0)
    #train_transactions = train_transactions.fillna("NA")
    app_transactions['playerId'] = app_transactions['playerId'].astype(int)
    player_df = pd.merge(player_df, 
                         app_transactions, 
                         left_on=['playerId'], right_on=['playerId'] ,how = "left")#,suffixes=(['','_Transit']))
    print('tran',len(player_df))
    #train_nextDayPlayerEngagement = pd.get_dummies(data=train_nextDayPlayerEngagement, columns=['typeCode'],prefix="transitionCode")

#[feature] player score box set features
    player_df = pd.merge(player_df, 
                         app_playerBoxScores[review_items_in_player], 
                         left_on=['playerId'], right_on=['playerId'] ,how = "left",suffixes=(['','_Player']))

    gc.collect()
    print('playerBox',len(player_df))

    #make team_df
#[Feature] standing
    standing_list = [
        'divisionId', 'teamId', 'teamName','streakCode', 
        'divisionRank', 'leagueRank', 'wildCardRank',
        'leagueGamesBack', 'sportGamesBack', 'divisionGamesBack', 'wins',
        'losses', 'pct', 'runsAllowed', 'runsScored', 'divisionChamp',      
        'divisionLeader', 'wildCardLeader', 'eliminationNumber',     
        'wildCardEliminationNumber', 'homeWins', 'homeLosses', 'awayWins',
        'awayLosses', 'lastTenWins', 'lastTenLosses', 'extraInningWins',
        'extraInningLosses', 'oneRunWins', 'oneRunLosses', 'divWins', 'divLosses', 'alWins', 'alLosses',
        'nlWins', 'nlLosses', 'xWinLossPct']

    app_standings = app_standings[standing_list]
    app_standings['StreakW'] = app_standings['streakCode'].str[0]
    app_standings['Streak_count'] = app_standings['streakCode'].str[1:].fillna(0).astype(int)
    
    
    #first_date dividchamp
    
    if first_data_swtich ==True:
        last_dividchamp = pd.DataFrame(columns = ['teamId','firstday_confirm_division','confirm_division_heat'])
    in_dividchamp = app_standings[app_standings['divisionChamp']==True]['teamId']
    in_dividchamp = pd.merge(in_dividchamp,last_dividchamp,left_on=['teamId'],right_on=['teamId'],how='left')
    in_dividchamp['confirm_division_heat'] = in_dividchamp['confirm_division_heat']/2
    in_dividchamp['confirm_division_heat'] = in_dividchamp['confirm_division_heat'].fillna(1)
    if len(in_dividchamp) > 0:
        in_dividchamp['firstday_confirm_division']=in_dividchamp.apply(lambda x: True if x['confirm_division_heat']==1 else False,axis=1)
    
    app_standings = pd.merge(app_standings,
                             in_dividchamp, 
                             left_on=['teamId'], right_on=['teamId'] ,how = "left")
                             
    
    #app_standings['firstday_confirm_division'] = app_standings['firstday_confirm_division'].fillna(False)
    #app_standings['confirm_division_heat']= app_standings['confirm_division_heat'].fillna(0)
    
    team_df = pd.concat([pd.DataFrame(),app_standings],)
    #team_df = pd.merge(team_df, train_teamBoxScores, left_on=['teamId','gameDate_DT'], right_on=['teamId','gameDate_DT'] ,how = "left",suffixes=(['_stand','_team']))

    
    team_df['teamId'] = team_df['teamId'].astype(int)
    #merge team to player
    player_df = pd.merge(player_df, 
                         team_df, 
                         left_on=['teamId_lastGameinMLB'], right_on=['teamId'] ,how = "left",suffixes=(['','_Team']))
    
    player_df['firstday_confirm_division'] = player_df['firstday_confirm_division'].fillna(False)
    player_df['confirm_division_heat']= player_df['confirm_division_heat'].fillna(0)

    
    print('standing',len(player_df))
    #[Feature] make game_df
    #what if player have 2 game in a day? do one more set and take the max() in test, but keep here first
    #display(app_games)
    app_games = app_games[[
        'gamePk', 'gameType', 'gameDate',
        'codedGameState', 'detailedGameState',
        'isTie', 'gameNumber', 'doubleHeader', 'dayNight', 'scheduledInnings',
        'gamesInSeries', 'seriesDescription', 'homeId',
        'homeWins', 'homeLosses', 'homeWinPct', 'homeWinner',
        'homeScore', 'awayId', 'awayWins',
        'awayLosses', 'awayWinPct', 'awayWinner', 'awayScore']]

    away_fea = [
        'gamePk', 'gameType',  'gameDate',
        'codedGameState', 'detailedGameState',
        'isTie', 'gameNumber', 'doubleHeader', 'dayNight', 'scheduledInnings',
        'gamesInSeries', 'seriesDescription', 'awayId', 'awayWins',
        'awayLosses', 'awayWinPct', 'awayWinner', 'awayScore','homeId',
        'homeWins', 'homeLosses', 'homeWinPct', 'homeWinner',
        'homeScore', ]

    regular_fea = [
        'gamePk', 'gameType', 'gameDate',
        'codedGameState', 'detailedGameState',
        'isTie', 'gameNumber', 'doubleHeader', 'dayNight', 'scheduledInnings',
        'gamesInSeries', 'seriesDescription', 'teamId',
        'teamWins', 'teamLosses', 'teamWinPct', 'teamWinner',
        'teamScore', 'oppId', 'oppWins',
        'oppLosses', 'oppWinPct', 'oppWinner', 'oppScore']

    home_app_games = pd.concat([pd.DataFrame(),app_games])
    home_app_games.columns = regular_fea
    home_app_games['home_away'] = 1

    away_app_games = pd.concat([pd.DataFrame(),app_games[away_fea]])
    away_app_games.columns = regular_fea
    away_app_games['home_away'] = 0
    
    app_games_team = pd.concat([home_app_games,away_app_games])
    del home_app_games
    del away_app_games
    gc.collect()
    app_games_team['gamePk'] = app_games_team['gamePk'].fillna(0)
    #for situation on All-star team 159 & 160
    app_games_team['teamId'] = app_games_team['teamId'].apply(lambda x: 159 if x == 160 else x)
    app_games_team['oppId'] = app_games_team['oppId'].apply(lambda x: 159 if x == 160 else x)
    
    
    #home view
    player_df = pd.merge(player_df, 
                         app_games_team, 
                         left_on=['gamePk','teamId_lastGameinMLB'], right_on=['gamePk','teamId'] ,how = "left",suffixes=(['','_Game']))
    print('games',len(player_df))
    
    #[Feature] Award gain
    #Cat group up
    if pd.isnull(test_df['awards']).iloc[0]:
        app_awards['playerId'] = app_awards['playerId'].fillna(0)
        app_awards['awardName'] = app_awards['awardName'].fillna("NA")
    app_awards['Award_1stword'] = app_awards['awardName'].str.split(' ').str[0]
    #app_awards['Award_1stword'] = app_awards['Award_1stword'].apply(lambda x: x if x.str.contains('|'.join(award_known_1stword),case=True,na=False,regex = True) else "Other")
    app_awards['Award_1stword_nonOther'] = app_awards['Award_1stword'].str.contains('|'.join(award_known_1stword),case=True,na=False,regex = True)
    app_awards['Award_1stword'] = app_awards.apply(lambda x: x['Award_1stword'] if x['Award_1stword_nonOther'] else "Other", axis=1)
    app_awards['Award_week'] = app_awards['awardName'].str.contains('of the week',case=False,na=False)
    app_awards['Award_month'] = app_awards['awardName'].str.contains('of the month',case=False,na=False)
    app_awards['Award_year'] = app_awards['awardName'].str.contains('of the year',case=False,na=False)
    app_awards['Award_AllStar'] = app_awards['awardName'].str.contains('All-Star',case=False,na=False)
    app_awards['Award_MVP'] = app_awards['awardName'].str.contains('MVP|Most Valuable Player|Championship',case=False,na=False,regex = True)
    app_awards['Award_Level'] = np.clip(app_awards['Award_week']+2*app_awards['Award_month']+3*app_awards['Award_year']+4*app_awards['Award_AllStar']+5*app_awards['Award_MVP'],0,5)
    award_need = ['playerId','Award_1stword','Award_Level']

    #group 1) count award in player in date, 2)max award_level, 1stword as mult if count > 1
    app_awards_player = app_awards[award_need].groupby(['playerId']).agg({'Award_1stword':['sum','count'],'Award_Level':'max'})
    app_awards_player.columns = ['_'.join(col) for col in app_awards_player.columns]
    app_awards_player['Award_1stword_sum'] = app_awards_player.apply(lambda x: "Mult" if x['Award_1stword_count']>1 else x['Award_1stword_sum'],axis = 1)
    app_awards_player = app_awards_player.reset_index()

    player_df = pd.merge(player_df, 
                         app_awards_player, 
                         left_on=['playerId'], right_on=['playerId'] ,how = "left",suffixes=(['','_Award']))

    print('award',len(player_df))

    
    #Create sets of category value from 0 to n
    #player: playerId, teamId, typeCode[transition],'fromTeamId'[transition],'toTeamId'[transition], statusCode ,[awards]
    #Team: gameType
    for feature in Cat_feature:
        player_df[feature] = player_df[feature].fillna("NA")
        player_df['label_'+feature] = player_df[feature].map(Cat_feature_MapDict[feature])
        #fill all new cat item as NA code
        if 'NA' in list(Cat_feature_MapDict[feature].keys()):
            player_df['label_'+feature] = player_df['label_'+feature].fillna(Cat_feature_MapDict[feature]["NA"])
        else:
            player_df['label_'+feature] = player_df['label_'+feature].fillna(0)
    gc.collect()
    
    

    #-----------------------------------Model Perdict------------------------------------------
    player_df['target1'] = np.clip(model1.predict(player_df[feature_colsM1]),0,100)
    player_df['target2_10log100'] = model2.predict(player_df[feature_colsM2])
    player_df['target3'] = np.clip(model3.predict(player_df[feature_colsM3]),0,100)
    player_df['target4_10log100'] = model4.predict(player_df[feature_colsM4])

    #prediction revert on T2 T4
    player_df['target2'] = np.clip(target_10log100_reverse(player_df['target2_10log100']),0,100)
    player_df['target4'] = np.clip(target_10log100_reverse(player_df['target4_10log100']),0,100)
    
    #-----------------------------------Submission & last date data record------------------------------------------
    #take max targets in same player as final targets
    player_df_FinalTargets = player_df[['playerId','target1','target2','target3','target4','target2_10log100','target4_10log100']].groupby('playerId').max().reset_index()    
    
    print('F_player',len(player_df_FinalTargets))
    print('sample1',len(sample_prediction_df))
    
    #Update targets
    sample_prediction_df[['date','playerId']] = sample_prediction_df['date_playerId'].str.split('_',expand=True)
    sample_prediction_df['playerId']= sample_prediction_df['playerId'].astype(int)
    sample_prediction_df = pd.merge(sample_prediction_df[['date_playerId','playerId']],
                                    player_df_FinalTargets[['playerId','target1','target2','target3','target4']],
                                    left_on=['playerId'], right_on=['playerId'] ,how = "left")
    print('sample2',len(sample_prediction_df))
    env.predict(sample_prediction_df[columns_store])

    #[Preiod record] record whole data set for next itr
    last_recorded_target_set =pd.concat([pd.DataFrame(),player_df_FinalTargets[['playerId','target1','target2_10log100','target3','target4_10log100']]])
    last_recorded_Cat = player_df[['playerId','teamId_lastGameinMLB','roster_status_lastGame']].groupby('playerId').last().reset_index()
    last_recorded_Cat['teamId_lastGameinMLB'] = last_recorded_Cat['teamId_lastGameinMLB'].replace("NA",np.nan)
    last_dividchamp= pd.concat([pd.DataFrame(),in_dividchamp])
    first_data_swtich = False
    gc.collect()
    
    #for submission check
    #player_df['date_DT'] = app_date
    #acc_record_target = pd.concat([acc_record_target,sample_prediction_df[columns_store]])
    #acc_record_x = pd.concat([acc_record_x,player_df])

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
2021-04-26 00:00:00
1player 2061
tran 2062
playerBox 2062
standing 2062
games 2062
award 2062
F_player 2061
sample1 1187
sample2 1187
2021-04-27 00:00:00
1player 2061
tran 2062
playerBox 2062
standing 2062
games 2062
award 2062
F_player 2061
sample1 1187
sample2 1187
2021-04-28 00:00:00
1player 2061
tran 2062
playerBox 2062
standing 2062
games 2062
award 2062
F_player 2061
sample1 1187
sample2 1187
2021-04-29 00:00:00
1player 2061
tran 2064
playerBox 2074
standing 2074
games 2074
award 2074
F_player 2061
sample1 1187
sample2 1187
2021-04-30 00:00:00
1player 2061
tran 2061
playerBox 2061
standing 2061
games 2061
award 2061
F_player 2061
sample1 1187
sample2 1187


In [10]:
#review function nan loadable

sample_prediction_df[columns_store]


,date_playerId,target1,target2,target3,target4
0,20210501_488726,2.666110,5.070983,0.032612,3.084068
1,20210501_605218,0.000789,0.256448,0.004616,2.117260
2,20210501_621563,0.112959,1.674976,0.008280,1.043393
3,20210501_670084,0.018730,0.423990,0.000711,0.408183
4,20210501_670970,0.003558,0.169380,0.019951,0.125054
...,...,...,...,...,...
1182,20210501_596049,0.000011,0.015746,0.000588,0.046211
1183,20210501_642851,0.000073,0.028499,0.000042,0.147654
1184,20210501_596071,0.000135,0.076962,0.000042,0.135630
1185,20210501_664901,0.002739,0.465343,0.001028,0.343107


In [11]:

#acc_record_x.to_csv('result_x1.csv')
#acc_record_target.to_csv('result in submission.csv')


In [12]:
#acc_record_x[(acc_record_x['teamId_lastGameinMLB']!="NA")&(acc_record_x['divisionRank'].isna())]
#(acc_record_x['divisionRank'].isna()).mean()


In [13]:
#player_df[player_df['teamId_lastGameinMLB']!="NA"].count()

In [14]:
#(aaplayer_df['teamId_lastGameinMLB'].isna()).mean()
#(aaplayer_df['divisionRank_Team'].isna()).mean()

In [15]:
#app_games_team['teamId']